In [1]:
from dotenv import load_dotenv

load_dotenv(override=True)

True

#### **First Concept- Model**

In [2]:
from autogen_ext.models.openai import OpenAIChatCompletionClient

model_client = OpenAIChatCompletionClient(model = 'gpt-4.1-nano')

#### **Second Concept- Messages**

In [3]:
from autogen_agentchat.messages import TextMessage

message = TextMessage(content="I love Paris!", source="user")
message

TextMessage(id='af834d50-a463-4dc6-91a6-f80174a3528d', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 7, 23, 12, 54, 4, 252888, tzinfo=datetime.timezone.utc), content='I love Paris!', type='TextMessage')

#### **Third Concept- Agent**

In [4]:
from autogen_agentchat.agents import AssistantAgent

agent = AssistantAgent(
    name="airline_agent",
    model_client=model_client,
    system_message="You are a helpful assistant for an airline. You give short, humorous answers",
    model_client_stream=True
)

Put it all together with on_messages

In [5]:
from autogen_core import CancellationToken

response = await agent.on_messages([message], cancellation_token=CancellationToken())
response.chat_message.content

"Paris is so Eiffel— I mean, special! Glad you're in love!"

`on_messages(...)` -> low level, direct message handling.
when we want to manually control the message flow.

*Aliter:* 
`agent.run()` -> High level, event driven execution loop

#### Let's make a local database of Ticket prices

In [6]:
import os
import sqlite3

# Deleting existing database file if it exists
if os.path.exists("tickets.db"):
    os.remove("tickets.db")

# Create the database and the table 
conn = sqlite3.connect("tickets.db")
c = conn.cursor()
c.execute("CREATE TABLE cities (city_name TEXT Primary Key, round_trip_price REAL)")
conn.commit()
conn.close()

In [ ]:
# Populate the database
def save_city_price(city_name, round_trip_price):
    conn = sqlite3.connect("tickets.db")
    c = conn.cursor()
    c.execute("REPLACE INTO cities (city_name, round_trip_price) VALUES (?, ?)", (city_name.lower(), round_trip_price))
    conn.commit()
    conn.close()

# Now some cities 
save_city_price("London", 299)
save_city_price("Paris", 399)
save_city_price("Rome", 499)
save_city_price("Madrid", 559)
save_city_price("Barcelona", 699)
save_city_price("Berlin", 999)

#### *Let's equip our agent with this tool!*

In [10]:
# Method to get price for a city

def get_city_price(city_name: str) -> float | None:
    """Get the round trip ticket price to travel to the city"""

    conn = sqlite3.connect("tickets.db")
    c = conn.cursor()
    c.execute("SELECT round_trip_price FROM cities WHERE city_name = ?", (city_name.lower(),))
    result = c.fetchone()
    conn.close()
    return result[0] if result else None

In [11]:
get_city_price("London")

299.0

One thing worth noticing is that, here we are not wrapping the python funtions in a Tool wrapper like we did in OpenAI Agents SDK and in LangGraph. We are just passing the function as it is.

In [ ]:
from autogen_agentchat.agents import AssistantAgent

smart_agent = AssistantAgent(
    name="smart_airline_agent",
    model_client=model_client,
    system_message="You are a helpful assistant for an airline. You give short, humorous answers along with the round trip ticket price.",
    tools=[get_city_price],
    model_client_stream=True,
    reflect_on_tool_use=True     # Lets agent think after using tools before answering
)

In [21]:
response = await smart_agent.on_messages([message], cancellation_token=CancellationToken())
response

Response(chat_message=TextMessage(id='86261e2a-5271-48b2-8e33-70677033cbf2', source='smart_airline_agent', models_usage=RequestUsage(prompt_tokens=0, completion_tokens=0), metadata={}, created_at=datetime.datetime(2025, 7, 23, 13, 9, 37, 646682, tzinfo=datetime.timezone.utc), content='Paris is always a good idea! Your round trip to the City of Lights is $399. Bon voyage!', type='TextMessage'), inner_messages=[ToolCallRequestEvent(id='dea980c9-377b-4d5e-a246-761444c81232', source='smart_airline_agent', models_usage=RequestUsage(prompt_tokens=0, completion_tokens=0), metadata={}, created_at=datetime.datetime(2025, 7, 23, 13, 9, 32, 972144, tzinfo=datetime.timezone.utc), content=[FunctionCall(id='call_jZpXw9ONUuSHkCqy1ocG53xU', arguments='{"city_name": "Paris"}', name='get_city_price')], type='ToolCallRequestEvent'), ToolCallExecutionEvent(id='d9e2246d-4f1c-48b2-b89f-3da9a3f6f181', source='smart_airline_agent', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 7, 23, 13, 

In [22]:
for inner_message in response.inner_messages:
    print(inner_message.content)

print(response.chat_message.content)

[FunctionCall(id='call_jZpXw9ONUuSHkCqy1ocG53xU', arguments='{"city_name": "Paris"}', name='get_city_price')]
[FunctionExecutionResult(content='399.0', name='get_city_price', call_id='call_jZpXw9ONUuSHkCqy1ocG53xU', is_error=False)]
Paris is always a good idea! Your round trip to the City of Lights is $399. Bon voyage!
